<a href="https://colab.research.google.com/github/p-tech/wbs-dm/blob/main/Cardinality/Cardinality_DB_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

# Connect to SQLite database (creates a file 'library.db' if not exists)
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

In [2]:
# 1️⃣ Create Tables
cursor.executescript("""
    -- Librarian Table (1:M with Borrower)
    CREATE TABLE IF NOT EXISTS Librarian (
        LibrarianID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Shift TEXT NOT NULL
    );

    -- Borrower Table (M:1 with Librarian)
    CREATE TABLE IF NOT EXISTS Borrower (
        BorrowerID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        ContactInfo TEXT,
        LibrarianID INTEGER,
        FOREIGN KEY (LibrarianID) REFERENCES Librarian(LibrarianID)
    );

    -- Library Branch Table (1:M with Book)
    CREATE TABLE IF NOT EXISTS LibraryBranch (
        BranchID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Location TEXT NOT NULL
    );

    -- Book Table (M:1 with Library Branch)
    CREATE TABLE IF NOT EXISTS Book (
        BookID INTEGER PRIMARY KEY AUTOINCREMENT,
        Title TEXT NOT NULL,
        Author TEXT NOT NULL,
        ISBN TEXT NOT NULL,
        BranchID INTEGER,
        FOREIGN KEY (BranchID) REFERENCES LibraryBranch(BranchID)
    );

    -- Loan_Record Table (M:M between Borrower and Book)
    CREATE TABLE IF NOT EXISTS Loan_Record (
        LoanID INTEGER PRIMARY KEY AUTOINCREMENT,
        BorrowerID INTEGER,
        BookID INTEGER,
        BorrowDate TEXT,
        ReturnDate TEXT,
        FOREIGN KEY (BorrowerID) REFERENCES Borrower(BorrowerID),
        FOREIGN KEY (BookID) REFERENCES Book(BookID)
    );
""")
conn.commit()

In [3]:
# 2️⃣ Insert Sample Data
cursor.executescript("""
    -- Insert Librarians
    INSERT INTO Librarian (Name, Shift) VALUES
        ('Sarah Johnson', 'Morning'),
        ('James Smith', 'Evening');

    -- Insert Borrowers
    INSERT INTO Borrower (Name, ContactInfo, LibrarianID) VALUES
        ('Alice Brown', 'alice@example.com', 1),
        ('Bob White', 'bob@example.com', 2);

    -- Insert Library Branches
    INSERT INTO LibraryBranch (Name, Location) VALUES
        ('Central Library', 'Main Street'),
        ('Westside Library', 'West Avenue');

    -- Insert Books
    INSERT INTO Book (Title, Author, ISBN, BranchID) VALUES
        ('Database Design', 'John Carter', '978-1234567890', 1),
        ('Digital Business', 'Emily Davis', '978-0987654321', 2);

    -- Insert Loan Records (M:M Relationship)
    INSERT INTO Loan_Record (BorrowerID, BookID, BorrowDate, ReturnDate) VALUES
        (1, 1, '2024-02-01', '2024-02-15'),
        (2, 1, '2024-02-20', '2024-03-05'),
        (1, 2, '2024-02-10', NULL);
""")
conn.commit()

In [6]:
# 1️⃣ Find Books Borrowed by a Specific Borrower
def get_borrowed_books(borrower_name):
    query = """
        SELECT Borrower.Name, Book.Title, Loan_Record.BorrowDate, Loan_Record.ReturnDate
        FROM Loan_Record
        JOIN Borrower ON Loan_Record.BorrowerID = Borrower.BorrowerID
        JOIN Book ON Loan_Record.BookID = Book.BookID
        WHERE Borrower.Name = ?;
    """
    cursor.execute(query, (borrower_name,))
    results = cursor.fetchall()
    return results

# 2️⃣ Count Number of Books Borrowed per Borrower
def count_books_per_borrower():
    query = """
        SELECT Borrower.Name, COUNT(Loan_Record.BookID) AS BooksBorrowed
        FROM Loan_Record
        JOIN Borrower ON Loan_Record.BorrowerID = Borrower.BorrowerID
        GROUP BY Borrower.Name;
    """
    cursor.execute(query)
    return cursor.fetchall()

# 3️⃣ Find Available Books in a Specific Library Branch
def available_books(branch_name):
    query = """
        SELECT Book.Title, Book.Author, LibraryBranch.Name AS Branch
        FROM Book
        JOIN LibraryBranch ON Book.BranchID = LibraryBranch.BranchID
        WHERE LibraryBranch.Name = ?;
    """
    cursor.execute(query, (branch_name,))
    return cursor.fetchall()

# 4️⃣ Find Borrowers and Their Assigned Librarians
def borrowers_librarians():
    query = """
        SELECT Borrower.Name AS Borrower, Librarian.Name AS AssignedLibrarian
        FROM Borrower
        JOIN Librarian ON Borrower.LibrarianID = Librarian.LibrarianID;
    """
    cursor.execute(query)
    return cursor.fetchall()

# 4️⃣ Run Queries
print("\n📚 Books Borrowed by Alice Brown:")
print(get_borrowed_books("Alice Brown"))

print("\n📊 Number of Books Borrowed per Borrower:")
print(count_books_per_borrower())

print("\n📖 Books Available in Central Library:")
print(available_books("Central Library"))

print("\n👤 Borrowers and Their Assigned Librarians:")
print(borrowers_librarians())

# Close the database connection
conn.close()


📚 Books Borrowed by Alice Brown:
[('Alice Brown', 'Database Design', '2024-02-01', '2024-02-15'), ('Alice Brown', 'Digital Business', '2024-02-10', None)]

📊 Number of Books Borrowed per Borrower:
[('Alice Brown', 2), ('Bob White', 1)]

📖 Books Available in Central Library:
[('Database Design', 'John Carter', 'Central Library')]

👤 Borrowers and Their Assigned Librarians:
[('Alice Brown', 'Sarah Johnson'), ('Bob White', 'James Smith')]
